In [1]:
include("load_s1slc_ann.jl");
include("load_pod.jl");
include("calc_sat_trajectory.jl");
include("approx_los.jl");
include("intersect.jl")
include("zero_doppler_bisect.jl")
include("ellipsoid2xyz.jl");

In [2]:
f = open("ann_path.txt")
path = readlines(f)
meta = load_s1slc_ann(path[1]);
f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],meta["t_0"]);

In [3]:
t_start = meta["t_start"]
t_stop = meta["t_stop"]
osv_poly, osv_mean, osv_std = calc_sat_trajectory(osv, t_sv, t_start, t_stop);

# Test point

In [4]:
lat_1 = (56 + 33/60 + 44.4528/(60*60)) * pi/180
lon_1 = (8 + 18/60 + 24.865/(60*60)) * pi/180
height_1 = 43.282;

point_test = ellipsoid2xyz(lat_1, lon_1, height_1)


time_test, range_test = zero_doppler_bisect(point_test, t_start, t_stop,
                                    osv_poly, osv_mean, osv_std);

In [5]:
osv_test = polyval_sv(osv_poly,time_test,osv_mean, osv_std);
theta_0 = meta["incidence_angle_mid"]*pi/180
x_sat_0 = osv_test[1:3]
v_sat_0 = osv_test[4:6];

#### dif test

In [6]:
#x_sat_0 .- osv_zd[1:3]

In [7]:
#v_sat_0 .- osv_zd[4:6]

In [8]:
x_0 = intersect(x_sat_0, approx_los(x_sat_0,v_sat_0,theta_0))

3-element Array{Float64,1}:
      3.485301583509802e6
 356325.1335164992       
      5.311889822091754e6

In [13]:
include("radar_solve.jl");

In [14]:
x_fun = radar_solve(range_test,height_1,x_0,osv_test[1:3],osv_test[4:6])

3-element Array{Float64,1}:
      3.4858248295596163e6
 508956.0100911942        
      5.299237308907142e6 

In [15]:
x_fun .- point_test

3-element Array{Float64,1}:
  0.0009693554602563381
 -1.0100193321704865e-6
 -0.000695202499628067 

In [19]:
semi_major_axis=6378137.
flattening=1/298.257223563
semi_minor_axis = semi_major_axis*(1 - flattening)
epsilon = (semi_major_axis/semi_minor_axis)^2  - 1 # second eccentricity squared
ecc_squared = flattening*(2-flattening)

0.0066943799901413165

In [20]:
# inputs
scale_fac = 1e-03;
MAX_ITER = 150; # max. number of iterations
eps = 1e-9;     # convergence epsilon

# inits
x_i_old = [0.;0.;0.]; 
x_i = x_0;
iter  = 1;

# scale
v_sat = v_sat_0 .*scale_fac;
#x_i = x_i .*[3.485824828590261e6, 508956.0100922042, 5.299237309602344e6];
x_i = x_i .*scale_fac;

x_sat = x_sat_0 .*scale_fac;
range_sat = range_sat_0 .*scale_fac;
a_plus_h = (semi_major_axis + height) .*scale_fac;
b_plus_h = (semi_minor_axis + height) .*scale_fac;




In [21]:
los = 0
while ((x_i - x_i_old)'*(x_i - x_i_old) > eps^2) & (iter < MAX_ITER)
    
    # Design matrix evaluated at previous solution
    los = x_i - x_sat
    fx_i = [v_sat'* los, 
        los' *los - range_sat^2, 
        ((x_i[1]^2 + x_i[2]^2) / a_plus_h^2 +(x_i[3] / b_plus_h)^2 - 1)];
    # Matrix of partial derivatives    
    dfx_i = vcat(v_sat',
            2*los',
            [2*x_i[1]/a_plus_h^2, 2*x_i[2]/a_plus_h^2,2*x_i[3]/b_plus_h^2]'); 
    
    # Solve linear system
    dx = dfx_i\(-fx_i)
    println(dx)
    # Update
    x_i_old = x_i;
    x_i += dx;
    iter += 1;
    
    
end

[1.43659, 136.339, -9.96898]
[-0.900169, 16.0568, -2.64483]
[-0.0131696, 0.234886, -0.0386921]
[-2.81938e-6, 5.02851e-5, -8.28331e-6]
[-1.94025e-14, 2.53193e-12, -2.28869e-13]


In [22]:
x_i *10^3 .- point_test

3-element Array{Float64,1}:
  0.0009693554602563381
 -1.0100193321704865e-6
 -0.000695202499628067 

In [23]:
iter

6

In [ ]:
while (x_i - x_i_1)'*(x_i - x_i_1) > eps^2 && iter < MAX_ITER

    % Design matrix evaluated at previous solution
    LOS_km = (Xi_km - XSat_km);
    fXi = [dot(VSat_km, LOS_km);...
           dot(LOS_km, LOS_km) - range_km^2;...
           (Xi_km(1)^2 + Xi_km(2)^2) / a_plus_h_km^2 + ...
           (Xi_km(3) / b_plus_h_km)^2 - 1;];

    % Matrix of partial derivatives    
    DfXi = [lying(VSat_km);...
            2*lying(LOS_km);
            2*Xi_km(1)/a_plus_h_km^2 2*Xi_km(2)/a_plus_h_km^2 2*Xi_km(3)/b_plus_h_km^2];      

    % Solve linear system
    DX = DfXi\(-fXi);
%     if iter == 1
%         fprintf('A = \n');
%         for i = 1 : size(DfXi,1)
%             for j = 1 : size(DfXi,2)
%                 fprintf('%.6e ', DfXi(i,j));
%             end
%             fprintf('\n');
%         end
%         fprintf('b = \n');
%         for i = 1 : length(fXi)
%             fprintf('%.6e ', -fXi(i));
%         end
%         fprintf('\n');
%         fprintf('X = \n');
%         for i = 1 : length(DX)
%             fprintf('%.6e ', DX(i));
%         end
%         fprintf('\n');
%         fprintf('rcond() = %.6e\n', rcond(DfXi));
%     end

    % Update solution
    Xi_1_km = Xi_km;
    Xi_km = Xi_1_km + DX;
    
    iter = iter + 1;
end
% Scale result back to meters
Xi = Xi_km / scaleFac;

if iter == MAX_ITER
    latP = nan;
    lonP = nan;
    hP = nan;
    warning('Maximum number of iterations reached without finding a solution.');
else
    [latP, lonP, hP] = xyz2ell2(Xi(1), Xi(2), Xi(3), elliSemiMajor, elliEccSquared);
    latP = rad2deg(latP);
    lonP = rad2deg(lonP);
    if lonP > 180
        lonP = lonP - 360;
    end
end